In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets as datasets
from torchvision import transforms as transforms
from torchvision.models import resnet34
from torchvision.models import ResNet34_Weights
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pickle as pkl

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [19]:
class LazyLoading(Dataset):
    def __init__(self, path, train=True, transform = None):
        self.transform = transform
        path = path+ ("train/" if train else "test/")
        self.pathX = path+"X/"
        self.pathY = path+"Y/"
        self.data = os.listdir(self.pathX)
    
    def __getitem__(self,idx):
        f = self.data[idx]
        img0 = cv2.imread(self.pathX+ f + "/rgb/0.png")
        img1 = cv2.imread(self.pathX+ f + "/rgb/1.png")
        img2 = cv2.imread(self.pathX+ f + "/rgb/2.png")
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        depth = np.load(self.pathX+f+"/depth.npy")
        # depth=cv2.normalize(depth, None, alpha=0, beta=1,
        #                      norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)   
        field_id = pkl.load(open(self.pathX+f+"/field_id.pkl","rb"))
        Y = np.load(self.pathY+f+".npy")
        return (img0, img1, img2, depth, field_id), Y*1000
    
    def __len__(self):
        return len(self.data)

In [20]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        # transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                            [0.229, 0.224, 0.225])
    ])

In [21]:
dataset = LazyLoading("./data/", transform=transform)

In [22]:
(img0,img1,img2,depth,field_id), Y = dataset[0]

In [23]:
train_loader = DataLoader(dataset, batch_size=64,shuffle=True)

In [24]:
res_model = resnet34(weights=ResNet34_Weights.DEFAULT)
res_model.eval()
res_model.float()
res_model.fc = nn.Linear(512,12)
weight = res_model.conv1.weight.clone()
res_model.conv1=  nn.Conv2d(12,64,kernel_size=7,stride=2,padding=3,bias=False)
with torch.no_grad():
    res_model.conv1.weight[:,:3]=weight
    res_model.conv1.weight[:,3]=res_model.conv1.weight[:,0]
res_model = res_model.to(device)

In [25]:
def train(epoch, model, optimizer):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        # rgbs = torch.stack((data[0][:,:,:,0],data[0][:,:,:,1],data[0][:,:,:,2],data[1][:,:,:,0],data[1][:,:,:,1],data[1][:,:,:,2],data[2][:,:,:,0],data[2][:,:,:,1],data[2][:,:,:,2]),1)
        data = torch.cat((data[0], data[1], data[2],data[3]),1)
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        mse_loss = nn.MSELoss()
        loss = mse_loss(output.float(), target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 40 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [26]:
optimizer = torch.optim.Adam(res_model.parameters())
for epoch in range(0, 50):
    train(epoch, res_model, optimizer)

Train Epoch: 0 [0/3396 (0%)]	Loss: 4457.271973
Train Epoch: 0 [2560/3396 (74%)]	Loss: 1690.103760
Train Epoch: 1 [0/3396 (0%)]	Loss: 1175.963379
Train Epoch: 1 [2560/3396 (74%)]	Loss: 406.470886
Train Epoch: 2 [0/3396 (0%)]	Loss: 303.996521
Train Epoch: 2 [2560/3396 (74%)]	Loss: 153.043060
Train Epoch: 3 [0/3396 (0%)]	Loss: 133.720947
Train Epoch: 3 [2560/3396 (74%)]	Loss: 94.644348
Train Epoch: 4 [0/3396 (0%)]	Loss: 82.932503
Train Epoch: 4 [2560/3396 (74%)]	Loss: 77.679688
Train Epoch: 5 [0/3396 (0%)]	Loss: 83.094009
Train Epoch: 5 [2560/3396 (74%)]	Loss: 44.962486
Train Epoch: 6 [0/3396 (0%)]	Loss: 47.614555
Train Epoch: 6 [2560/3396 (74%)]	Loss: 29.526920
Train Epoch: 7 [0/3396 (0%)]	Loss: 40.342525
Train Epoch: 7 [2560/3396 (74%)]	Loss: 24.690664
Train Epoch: 8 [0/3396 (0%)]	Loss: 24.181442
Train Epoch: 8 [2560/3396 (74%)]	Loss: 38.084167
Train Epoch: 9 [0/3396 (0%)]	Loss: 30.354614
Train Epoch: 9 [2560/3396 (74%)]	Loss: 28.457836
Train Epoch: 10 [0/3396 (0%)]	Loss: 29.563353
Trai

In [27]:
test_data = torch.load("./data/testX.pt")
file_ids = test_data[-1]
depths = test_data[1]
rgbs = test_data[0]

In [28]:
test_img_data = torch.cat((rgbs[:,0],rgbs[:,1],rgbs[:,2],depths),dim=1)

In [29]:
split_test = torch.split(test_img_data,50,dim=0)

In [30]:
preds = []
for data in split_test:
    output = res_model(data.to("cuda"))
    preds.append(output.cpu().detach().numpy())

In [31]:
import pandas as pd

outfile = 'submission.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
# preds = []

# test_data = torch.load('./test/test/testX.pt')
# file_ids = test_data[-1]
# rgb_data = test_data[0]
# model.eval()

# for i, data in enumerate(rgb_data):   
#     # Please remember to modify this loop, input and output based on your model/architecture
#     output = model(data[:1, :, :, :].to('cuda'))
#     preds.append(output[0].cpu().detach().numpy())

df = pd.concat([pd.DataFrame(file_ids), pd.DataFrame.from_records(np.concatenate(preds)/1000)], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

Written to csv file submission.csv
